In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed (1)

In [39]:
word_to_ix = {"hello":0, "world":1, "C++":2}
embeds = nn.Embedding (3, 5)

In [40]:
lookup_tensor = torch.tensor ([word_to_ix["hello"]], dtype = torch.long)
lookup_tensor = torch.tensor ([word_to_ix["C++"]], dtype = torch.long)

In [41]:
hello_embed = embeds(lookup_tensor)

In [42]:
print (hello_embed)

tensor([[-0.8210,  1.2519,  0.0883,  0.8994, -1.7052]],
       grad_fn=<EmbeddingBackward>)


In [43]:
CONTEXT_SIZE = 3
EMBEDDING_DIM = 10

In [44]:
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

In [45]:
trigrams = [([test_sentence[i], test_sentence[i+1], test_sentence[i+2]], test_sentence[i+3])
            for i in range (len(test_sentence)-3)]

In [46]:
trigrams[:3]

[(['When', 'forty', 'winters'], 'shall'),
 (['forty', 'winters', 'shall'], 'besiege'),
 (['winters', 'shall', 'besiege'], 'thy')]

In [47]:
vocab = set (test_sentence)

In [48]:
word_to_ix = {word : i for i, word in enumerate(vocab)}

In [49]:
class NGramLanguageModeler (nn.Module):
    def __init__ (self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embedding_dim = nn.Embedding (vocab_size, embedding_dim)
        self.linear1 = nn.Linear (context_size*embedding_dim, 128)
        self.linear2 = nn.Linear (128, vocab_size)
    
    def forward (self, inputs):
        embeds = self.embedding_dim (inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2 (out)
        log_probs = F.log_softmax (out, dim = 1)
        return log_probs

In [50]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD (model.parameters(), lr = 0.001)

In [51]:
for epoch in range (10):
    total_loss = 0
    for context, target in trigrams:
        # 1 准备好进入模型的数据，例如将单词转换成整数索引，并将其封装在变量中
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype = torch.long)
        # 2 回调torch累成梯度
        model.zero_grad()
        # 3 得到单词的概率值
        log_probs = model(context_idxs)
        # 4 计算损失函数
        loss = loss_function (log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))
        # 5 反向传播更新梯度
        loss.backward()
        optimizer.step()
        
        total_loss = loss.item()
    losses.append (total_loss)
print (losses)

[4.632925033569336, 4.619664669036865, 4.606525421142578, 4.593371391296387, 4.580132961273193, 4.566908836364746, 4.55371618270874, 4.540586471557617, 4.527451992034912, 4.514431476593018]
